In [1]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

In [3]:
pop_2010_2019 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/pop_est_2010-2019_fl.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Alachua,252475,255456,259052,263959,266309,268851,269043
1,Baker,27009,27122,27355,27884,28254,28353,29210
2,Bay,174596,178289,181488,183634,184736,186240,174705
3,Bradford,26802,26552,26748,26740,27142,27752,28201
4,Brevard,550255,555548,565746,576874,587769,595203,601942


In [4]:
pop_2020_2021 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/pop_est_2020-2021_fl.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Alachua,278684,279238
1,Baker,28309,28715
2,Bay,174461,179168
3,Bradford,28364,28540
4,Brevard,608007,616628


In [5]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined.drop(index=pop_combined.index[-1],axis=0,inplace=True)
pop_combined.head()

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Alachua,252475,255456,259052,263959,266309,268851,269043,278684,279238
1,Baker,27009,27122,27355,27884,28254,28353,29210,28309,28715
2,Bay,174596,178289,181488,183634,184736,186240,174705,174461,179168
3,Bradford,26802,26552,26748,26740,27142,27752,28201,28364,28540
4,Brevard,550255,555548,565746,576874,587769,595203,601942,608007,616628


In [6]:
metros = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_fl = metros_df[metros_df['State Name']=='Florida']
metros_fl.head()

,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
0,"Gainesville, FL",Metropolitan Statistical Area,Alachua,Florida
14,"Jacksonville, FL",Metropolitan Statistical Area,Baker,Florida
20,"Panama City, FL",Metropolitan Statistical Area,Bay,Florida
34,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area,Brevard,Florida
36,"Miami-Fort Lauderdale-Pompano Beach, FL",Metropolitan Statistical Area,Broward,Florida


In [7]:
file ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2014.pdf'

In [8]:
new_table=[]
pdf = pdfplumber.open(file)
table_p1=pdf.pages[49].extract_text()
table_p1 = table_p1[102:]
table_p1 = table_p1[:-8]
table_p1 = table_p1.split("\n")
for i in table_p1:
    t={}
    i = i.rsplit(" ",4)
    t = i
    new_table.append(t)

In [9]:
pdf = pdfplumber.open(file)
table_p2=pdf.pages[50].extract_text()
table_p2 = table_p2[23:]
table_p2 = table_p2[:-8]
table_p2 = table_p2.split("\n")
for i in table_p2:
    t={}
    i = i.rsplit(" ",4)
    t = i
    new_table.append(t)

In [10]:
pdf = pdfplumber.open(file)
table_p3=pdf.pages[51].extract_text()
table_p3 = table_p3[23:]
table_p3 = table_p3[:-8]
table_p3 = table_p3.split("\n")
for i in table_p3:
    t={}
    i = i.rsplit(" ",4)
    t = i
    new_table.append(t)

In [11]:
fl_df = pd.DataFrame(new_table)
fl_df = fl_df.drop([3,4], axis=1)
fl_df.columns=['County','total_crashes_2013','total_crashes_2014']
fl_df.drop(index=fl_df.index[-1],axis=0,inplace=True)
fl_df

,County,total_crashes_2013,total_crashes_2014
0,Alachua,"4,651","4,193"
1,Baker,293,246
2,Bay,"3,514","3,337"
3,Bradford,220,274
4,Brevard,"7,079","7,661"
5,Broward,"32,595","34,833"
6,Calhoun,152,134
7,Charlotte,"2,091","2,241"
8,Citrus,"1,360","1,471"
9,Clay,"2,434","2,594"


In [12]:
file_2014_2016 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2016.pdf'

In [13]:
pdf = pdfplumber.open(file_2014_2016)
table=pdf.pages[35].extract_table()
table = table[1:]
table

[['Alachua', 'Total Crashes', '5,980', '5,027', '4,193', '18.96%', '5,067'],
 [None, 'Total Fatalities', '46', '37', '30', '24.32%', '38'],
 [None, 'Total Injuries', '4,398', '3,826', '3,113', '14.95%', '3,779'],
 [None, 'Total', '10,424.00', '8,890.00', '7,336.00', '17.26%', '8,883.00'],
 ['Baker', 'Total Crashes', '381', '373', '246', '2.14%', '333'],
 [None, 'Total Fatalities', '13', '12', '2', '8.33%', '9'],
 [None, 'Total Injuries', '352', '286', '264', '23.08%', '301'],
 [None, 'Total', '746.00', '671.00', '512.00', '11.18%', '643.00'],
 ['Bay', 'Total Crashes', '4,089', '3,946', '3,337', '3.62%', '3,791'],
 [None, 'Total Fatalities', '38', '32', '30', '18.75%', '33'],
 [None, 'Total Injuries', '2,290', '2,317', '2,023', '-1.17%', '2,210'],
 [None, 'Total', '6,417.00', '6,295.00', '5,390.00', '1.94%', '6,034.00'],
 ['Bradford', 'Total Crashes', '362', '348', '274', '4.02%', '328'],
 [None, 'Total Fatalities', '6', '10', '4', '-40.00%', '7'],
 [None, 'Total Injuries', '243', '241'

In [14]:
pdf = pdfplumber.open(file_2014_2016)
table=pdf.pages[37].extract_table()
table = table[1:]
list_length = len(table)
paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
if list_length % 3 == 2:
   paired_list.append(table[list_length-1])
print(paired_list)    

[['Gilchrist', 'Total Crashes', '194', '176', '160', '10.23%', '177', None, 'Total Fatalities', '5', '3', '3', '66.67%', '4', None, 'Total Injuries', '123', '126', '109', '-2.38%', '119'], ['Glades', 'Total Crashes', '178', '155', '139', '14.84%', '157', None, 'Total Fatalities', '2', '14', '3', '-85.71%', '6', None, 'Total Injuries', '97', '114', '96', '-14.91%', '102'], ['Gulf', 'Total Crashes', '181', '140', '137', '29.29%', '153', None, 'Total Fatalities', '1', '2', '2', '-50.00%', '2', None, 'Total Injuries', '119', '81', '98', '46.91%', '99'], ['Hamilton', 'Total Crashes', '269', '253', '177', '6.32%', '233', None, 'Total Fatalities', '5', '5', '6', '0.00%', '5', None, 'Total Injuries', '159', '151', '129', '5.30%', '146'], ['Hardee', 'Total Crashes', '416', '382', '333', '8.90%', '377', None, 'Total Fatalities', '5', '2', '10', '150.00%', '6', None, 'Total Injuries', '334', '292', '256', '14.38%', '294'], ['Hendry', 'Total Crashes', '331', '270', '254', '22.59%', '285', None, 'T

In [15]:
df_test = pd.DataFrame(paired_list)
df_test = df_test.drop([5,6,7,12,13,14,19,20],axis=1)
df_test

,0,1,2,3,4,8,9,10,11,15,16,17,18
0,Gilchrist,Total Crashes,194,176,160,Total Fatalities,5,3,3,Total Injuries,123,126,109
1,Glades,Total Crashes,178,155,139,Total Fatalities,2,14,3,Total Injuries,97,114,96
2,Gulf,Total Crashes,181,140,137,Total Fatalities,1,2,2,Total Injuries,119,81,98
3,Hamilton,Total Crashes,269,253,177,Total Fatalities,5,5,6,Total Injuries,159,151,129
4,Hardee,Total Crashes,416,382,333,Total Fatalities,5,2,10,Total Injuries,334,292,256
5,Hendry,Total Crashes,331,270,254,Total Fatalities,19,10,8,Total Injuries,273,235,228
6,Hernando,Total Crashes,"2,142","2,153","1,981",Total Fatalities,25,35,20,Total Injuries,"2,093","2,052","1,764"
7,Highlands,Total Crashes,"1,329","1,218","1,067",Total Fatalities,21,22,21,Total Injuries,978,836,740
8,Hillsborough,Total Crashes,"27,660","25,967","23,024",Total Fatalities,228,190,158,Total Injuries,"21,380","19,889","17,561"
9,Holmes,Total Crashes,284,259,222,Total Fatalities,9,9,5,Total Injuries,158,141,169


In [16]:
df_test_melt_2016 = pd.melt(df_test,id_vars=[2,9,16],value_vars=[0])
df_test_melt_2016 = df_test_melt_2016.drop(['variable'],axis=1)
df_test_melt_2016.head()

,2,9,16,value
0,194,5,123,Gilchrist
1,178,2,97,Glades
2,181,1,119,Gulf
3,269,5,159,Hamilton
4,416,5,334,Hardee


In [17]:
df_test_melt_2015 = pd.melt(df_test,id_vars=[3,10,17],value_vars=[0])
df_test_melt_2015 = df_test_melt_2015.drop(['variable'],axis=1)
df_test_melt_2015.head()

,3,10,17,value
0,176,3,126,Gilchrist
1,155,14,114,Glades
2,140,2,81,Gulf
3,253,5,151,Hamilton
4,382,2,292,Hardee


In [18]:
df_test_melt_2014 = pd.melt(df_test,id_vars=[4,11,18],value_vars=[0])
df_test_melt_2014 = df_test_melt_2014.drop(['variable'],axis=1)
df_test_melt_2014.head()

,4,11,18,value
0,160,3,109,Gilchrist
1,139,3,96,Glades
2,137,2,98,Gulf
3,177,6,129,Hamilton
4,333,10,256,Hardee


### Extracting data for years 2014 thru 2016 and combining with pop along with Metropolitans and Micropolitans

In [19]:
fl_df = pd.DataFrame()
pages = list(range(35,42))
pdf = pdfplumber.open(file_2014_2016)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df = fl_df.append(df)

In [20]:
fl_df = fl_df.drop([5,6,7,12,13,14,19,20],axis=1)
fl_df.head()

,0,1,2,3,4,8,9,10,11,15,16,17,18
0,Alachua,Total Crashes,"5,980","5,027","4,193",Total Fatalities,46,37,30,Total Injuries,"4,398","3,826","3,113"
1,Baker,Total Crashes,381,373,246,Total Fatalities,13,12,2,Total Injuries,352,286,264
2,Bay,Total Crashes,"4,089","3,946","3,337",Total Fatalities,38,32,30,Total Injuries,"2,290","2,317","2,023"
3,Bradford,Total Crashes,362,348,274,Total Fatalities,6,10,4,Total Injuries,243,241,219
4,Brevard,Total Crashes,"8,752","8,387","7,661",Total Fatalities,98,82,65,Total Injuries,"6,564","6,130","5,386"


In [21]:
fl_df_melt_2016 = pd.melt(fl_df,id_vars=[2,9,16],value_vars=[0])
fl_df_melt_2016 = fl_df_melt_2016.drop(['variable'],axis=1)
fl_df_melt_2016 = fl_df_melt_2016[['value',9, 16,2]]
fl_df_melt_2016.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2016.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,46,"4,398","5,980"
1,Baker,13,352,381
2,Bay,38,"2,290","4,089"
3,Bradford,6,243,362
4,Brevard,98,"6,564","8,752"
5,Broward,245,"25,361","41,755"
6,Calhoun,3,120,194
7,Charlotte,30,"1,649","2,721"
8,Citrus,24,"1,232","1,682"
9,Clay,26,"1,857","3,053"


In [22]:
df_combined_fl = pd.merge(fl_df_melt_2016,pop_combined[['County', 2016]],on="County",how='left')
df_combined_fl_2016 = pd.merge(df_combined_fl,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2016.head()

,County,fatalaties,injuries,total_crashes,2016,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,46,"4,398","5,980",263959.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,13,352,381,27884.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,38,"2,290","4,089",183634.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,6,243,362,26740.0,NaN,NaN
4,Brevard,98,"6,564","8,752",576874.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [23]:
fl_df_melt_2015 = pd.melt(fl_df,id_vars=[10,17,3],value_vars=[0])
fl_df_melt_2015 = fl_df_melt_2015.drop(['variable'],axis=1)
fl_df_melt_2015 = fl_df_melt_2015[['value',10,17,3]]
fl_df_melt_2015.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2015.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,37,"3,826","5,027"
1,Baker,12,286,373
2,Bay,32,"2,317","3,946"
3,Bradford,10,241,348
4,Brevard,82,"6,130","8,387"
5,Broward,221,"23,473","38,409"
6,Calhoun,4,118,180
7,Charlotte,23,"1,616","2,556"
8,Citrus,29,"1,124","1,502"
9,Clay,33,"1,727","2,817"


In [24]:
df_combined_fl = pd.merge(fl_df_melt_2015,pop_combined[['County', 2015]],on="County",how='left')
df_combined_fl_2015 = pd.merge(df_combined_fl,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2015.head()

,County,fatalaties,injuries,total_crashes,2015,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,37,"3,826","5,027",259052.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,12,286,373,27355.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,32,"2,317","3,946",181488.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,10,241,348,26748.0,NaN,NaN
4,Brevard,82,"6,130","8,387",565746.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [25]:
fl_df_melt_2014 = pd.melt(fl_df,id_vars=[11,18,4],value_vars=[0])
fl_df_melt_2014 = fl_df_melt_2014.drop(['variable'],axis=1)
fl_df_melt_2014 = fl_df_melt_2014[['value',11,18,4]]
fl_df_melt_2014.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2014.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,30,"3,113","4,193"
1,Baker,2,264,246
2,Bay,30,"2,023","3,337"
3,Bradford,4,219,274
4,Brevard,65,"5,386","7,661"
5,Broward,173,"22,154","34,833"
6,Calhoun,2,94,134
7,Charlotte,19,"1,290","2,241"
8,Citrus,26,"1,099","1,471"
9,Clay,18,"1,784","2,594"


In [26]:
df_combined_fl = pd.merge(fl_df_melt_2014,pop_combined[['County', 2015]],on="County",how='left')
df_combined_fl_2014 = pd.merge(df_combined_fl,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2014.head()

,County,fatalaties,injuries,total_crashes,2015,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,30,"3,113","4,193",259052.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,2,264,246,27355.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,30,"2,023","3,337",181488.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,4,219,274,26748.0,NaN,NaN
4,Brevard,65,"5,386","7,661",565746.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [27]:
df_combined_fl_2014.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2014_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2015.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2015_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2016.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2016_FL.xlsx', engine='xlsxwriter',index=False)

In [28]:
file_2017 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2017.pdf'

In [29]:
# pdf = pdfplumber.open(file_2017)
# table=pdf.pages[35].extract_table()
# table = table[1:]
# list_length = len(table)
# paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
# paired_list

In [30]:
fl_df = pd.DataFrame()
pages = list(range(36,43))
pdf = pdfplumber.open(file_2017)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df = fl_df.append(df)

In [31]:
fl_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Clay,Total Injuries,"1,804","1,857","1,727",-2.85%,"1,796",None,,,,,,,Collier,Total Crashes,"5,135","5,273","4,632",-2.62%\n-34.29%\n-2.29%,"5,013\n36\n2,779"
1,None,Total Injuries,"2,819","2,885","2,633",None,None,None,,,,,,,Columbia,Total Crashes,"1,490","1,527","1,362",-2.42%\n0.00%\n-3.87%,"1,460\n21\n1,121"
2,None,Total Injuries,"1,117","1,162","1,084",None,None,None,,,,,,,DeSoto,Total Crashes,516,453,408,13.91%\n36.36%\n28.24%,459\n14\n337
3,None,Total Injuries,386,301,323,None,None,None,,,,,,,Dixie,Total Crashes,151,160,159,-5.63%\n100.00%\n25.00%,157\n4\n109
4,None,Total Injuries,120,96,110,None,None,None,,,,,,,Duval,Total Crashes,"24,734","24,107","22,423",2.60%\n-3.21%\n5.67%,"23,755\n147\n14,278"
5,None,Total Injuries,"14,850","14,053","13,931",None,None,None,,,,,,,Escambia,Total Crashes,"6,627","7,075","6,733",-6.33%\n10.87%\n-19.87%,"6,812\n49\n5,511"
6,None,Total Injuries,"4,686","5,848","6,000",None,None,None,,,,,,,Flagler,Total Crashes,"1,190","1,174","1,377",1.36%\n32.00%\n-2.66%,"1,247\n23\n886"
7,None,Total Injuries,806,828,"1,023",None,None,None,,,,,,,Franklin,Total Crashes,47,115,135,-59.13%\n-80.00%\n-56.25%,99\n2\n66
8,None,Total Injuries,35,80,84,None,None,None,,,,,,,Gadsden,Total Crashes,669,680,590,-1.62%\n5.56%\n-14.64%,646\n16\n430
9,None,Total Injuries,414,485,392,None,None,None,,,,,,,Gilchrist,Total Crashes,189,194,176,-2.58%,186


### Extracting data for years 2018 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [32]:
file_2018_2020 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2020.pdf'

In [33]:
fl_df = pd.DataFrame()
pages = list(range(38,45))
pdf = pdfplumber.open(file_2018_2020)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df = fl_df.append(df)

In [34]:
fl_df = fl_df.drop([5,6,7,12,13,14,19,20],axis=1)
fl_df.head()

,0,1,2,3,4,8,9,10,11,15,16,17,18
0,Alachua,Total Crashes,"4,617","5,878","5,846",Total Fatalities,48,72,51,Total Injuries,"3,266","4,237","4,415"
1,Baker,Total Crashes,420,438,400,Total Fatalities,4,7,5,Total Injuries,293,341,354
2,Bay,Total Crashes,"3,352","4,862","4,549",Total Fatalities,35,32,26,Total Injuries,"1,869","2,498","2,360"
3,Bradford,Total Crashes,329,412,449,Total Fatalities,10,12,9,Total Injuries,192,282,252
4,Brevard,Total Crashes,"8,395","9,207","9,233",Total Fatalities,81,84,85,Total Injuries,"5,768","6,625","6,911"


In [35]:
fl_df_melt_2020 = pd.melt(fl_df,id_vars=[2,9,16],value_vars=[0])
fl_df_melt_2020 = fl_df_melt_2020.drop(['variable'],axis=1)
fl_df_melt_2020 = fl_df_melt_2020[['value',9, 16,2]]
fl_df_melt_2020.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2020.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,48,"3,266","4,617"
1,Baker,4,293,420
2,Bay,35,"1,869","3,352"
3,Bradford,10,192,329
4,Brevard,81,"5,768","8,395"
5,Broward,268,"18,984","34,050"
6,Calhoun,3,120,198
7,Charlotte,26,"1,369","2,185"
8,Citrus,38,"1,194","1,685"
9,Clay,24,"1,502","2,511"


In [36]:
df_combined_fl = pd.merge(fl_df_melt_2020,pop_combined[['County', 2020]],on="County",how='left')
df_combined_fl_2020 = pd.merge(df_combined_fl,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2020.head()

,County,fatalaties,injuries,total_crashes,2020,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,48,"3,266","4,617",278684.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,4,293,420,28309.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,35,"1,869","3,352",174461.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,10,192,329,28364.0,NaN,NaN
4,Brevard,81,"5,768","8,395",608007.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [37]:
fl_df_melt_2019 = pd.melt(fl_df,id_vars=[10,17,3],value_vars=[0])
fl_df_melt_2019 = fl_df_melt_2019.drop(['variable'],axis=1)
fl_df_melt_2019 = fl_df_melt_2019[['value',10,17,3]]
fl_df_melt_2019.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2019.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,72,"4,237","5,878"
1,Baker,7,341,438
2,Bay,32,"2,498","4,862"
3,Bradford,12,282,412
4,Brevard,84,"6,625","9,207"
5,Broward,214,"23,671","41,114"
6,Calhoun,4,148,213
7,Charlotte,22,"1,418","2,386"
8,Citrus,22,"1,410","1,774"
9,Clay,26,"1,669","2,694"


In [38]:
df_combined_fl = pd.merge(fl_df_melt_2019,pop_combined[['County', 2019]],on="County",how='left')
df_combined_fl_2019 = pd.merge(df_combined_fl,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2019.head()

,County,fatalaties,injuries,total_crashes,2019,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,72,"4,237","5,878",269043.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,7,341,438,29210.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,32,"2,498","4,862",174705.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,12,282,412,28201.0,NaN,NaN
4,Brevard,84,"6,625","9,207",601942.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [39]:
fl_df_melt_2018 = pd.melt(fl_df,id_vars=[11,18,4],value_vars=[0])
fl_df_melt_2018 = fl_df_melt_2018.drop(['variable'],axis=1)
fl_df_melt_2018 = fl_df_melt_2018[['value',11,18,4]]
fl_df_melt_2018.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2018.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,51,"4,415","5,846"
1,Baker,5,354,400
2,Bay,26,"2,360","4,549"
3,Bradford,9,252,449
4,Brevard,85,"6,911","9,233"
5,Broward,237,"24,644","41,082"
6,Calhoun,8,112,206
7,Charlotte,27,"1,582","2,734"
8,Citrus,37,"1,413","1,813"
9,Clay,19,"1,737","2,849"


In [40]:
df_combined_fl = pd.merge(fl_df_melt_2018,pop_combined[['County', 2018]],on="County",how='left')
df_combined_fl_2018 = pd.merge(df_combined_fl,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2018.head()

,County,fatalaties,injuries,total_crashes,2018,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,51,"4,415","5,846",268851.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,5,354,400,28353.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,26,"2,360","4,549",186240.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,9,252,449,27752.0,NaN,NaN
4,Brevard,85,"6,911","9,233",595203.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [41]:
df_combined_fl_2018.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2018_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2019.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2019_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2020.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2020_FL.xlsx', engine='xlsxwriter',index=False)